In [ ]:
import os
from hostdesigner.host import Host
from hostdesigner.visualize import show
from hostdesigner.run import hd_run
from hostdesigner.output import read_hdo, find_results

## Running HostDesigner
#### Selecting run directory

In [ ]:
hd_path = os.path.join(os.getcwd(), 'HD_3.0')
ex_path = os.path.join(hd_path, '03_Examples')

mol1_path = os.path.join(ex_path, 'case1', 'hosta')
mol2_path = os.path.join(ex_path, 'case1', 'hostb')

#### Visualizing host structures

In [ ]:
h1 = Host(mol1_path)

h2 = Host(mol2_path, run_type='LINKER')

show(h1, h2, rename='F')

#### Running examples: Case 1

In [ ]:
case1_path = os.path.join(ex_path, 'case1')
hd_run(case1_path)

## Reading results

In [ ]:
results_dir = os.path.join(ex_path, 'case1')
results = find_results(results_dir)

hdo1 = read_hdo(results['hdo'][1]) # out_1.hdo -> sorted by RMSD
hdo2 = read_hdo(results['hdo'][0]) # out_2.hdo -> sorted by energy

#### Listing results sorted by RMSD

In [ ]:
hdo1.tabulate(structures=5)

#### Listing results sorted by Energy

In [ ]:
hdo2.tabulate(structures=5)

#### Visualizing results (8 structures | 4 in x-axis)

In [ ]:
hdo1.show(8, div=4)

#### Sorting structures according to number of atoms

In [ ]:
hdo1_sorted = hdo1.sort(var='n_atoms', table=True, structures=5)

In [ ]:
hdo1_sorted.show(8, div=4)